In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from py3SEB_workshop.functions.biophysical import get_diffuse_radiation_6S, build_soil_database, SRF_LIBRARY, S2_BANDS
from pypro4sail import machine_learning_regression as inv

In [ ]:
from Py6S import *
SixS.test()

In [ ]:
# bands to use in generating LUT and inversion
S2_BANDS = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12']
# Stack spectral bands
srf = []
srf_file = SRF_LIBRARY / f'Sentinel2A.txt'
srfs = np.genfromtxt(srf_file, dtype=None, names=True)
for band in S2_BANDS:
    srf.append(srfs[band])

# open as pandas dataframe
srf_df = pd.read_csv(srf_file, sep = '\t')

band_names = srf_df[S2_BANDS].columns

# plot spectral response function
colormap = plt.cm.rainbow # Choose a colormap
# get color within colormap range for each band (depends on number of bands)
colors = [colormap(x / (len(band_names) - 1)) for x in range(len(band_names))]

# Show spectral response function (SRF) curves
plt.figure(figsize=(9, 5))
plt.title(f'Spectral Response Function (SRF) - Sentinel-2', fontsize=14)
plt.xlabel('Wavelength (nm)', fontsize=12)
plt.xlim(400, 2500)
plt.ylabel('Relative Response (-)', fontsize=12)
plt.ylim(0, 1)
plt.grid(True)
i = 0
for band in band_names:
    plt.plot(srf_df['SR_WL'], srf_df[band], color=colors[i], label = f'{str(band)}')
    i += 1

plt.legend(loc='lower right', ncol=5)
plt.show()

In [ ]:
## Build simulated Sentinel-2-like Look-Up-Table (LUT)

In [ ]:
import Py6S as sixs

date=date_obj
altitude=0.1
wls_step=10
n_jobs=None

s = sixs.SixS()

s.atmos_profile = sixs.AtmosProfile.PredefinedType(
sixs.AtmosProfile.MidlatitudeSummer)

s.aeroprofile = sixs.AeroProfile.PredefinedType(sixs.AeroProfile.Continental)

s.ground_reflectance = sixs.GroundReflectance.HomogeneousLambertian(0)

if np.isfinite(wvp) and wvp > 0:
    s.atmos_profile = sixs.AtmosProfile.UserWaterAndOzone(wvp, 0.9)

if np.isfinite(aot) and aot > 0:
    s.aot550 = aot

s.geometry.solar_a = saa
s.geometry.view_z = 0
s.geometry.view_a = 0
s.geometry.day = date.day
s.geometry.month = date.month

s.altitudes.set_target_custom_altitude(altitude)
s.wavelength = sixs.Wavelength(0.4, 2.5)

wls = np.arange(400, 2501)
wls_sim = np.arange(400, 2501, wls_step)

wv, res = sixs.SixSHelpers.Wavelengths.run_wavelengths(s,
                                                           wls_sim / 1000.,
                                                           verbose=False,
                                                           n=n_jobs)

In [ ]:
print(f"Building {np.size(params_orig['bs'])} PROSPECTD+4SAIL simulations")
soil_spectrum = build_soil_database(params_orig["bs"])
print('Done!')

In [ ]:
n_simulations = 40000

# parameter names
OBJ_PARAM_NAMES = ["Cab", "Car", "Cm", "Cw", "Ant", "Cbrown",
                   "LAI", "leaf_angle"]
# parameter info
PARAM_PROPS = {"Cab": ["Chlorophyll a+b", r"$\mu g\,cm^{-2}$", 1],
               "Car": ["Carotenoids", r"$\mu g\,cm^{-2}$", 1],
               "Cm": ["Dry matter", r"$g\,cm^{-2}$", 3],
               "Cw": ["Water content", r"$g\,cm^{-2}$", 3],
               "Ant": ["Antocyanins", r"$\mu g\,cm^{-2}$", 1],
               "Cbrown": ["Brown pigments", r"$-$", 1],
               "LAI": ["Leaf Area Index", r"$m^{2}\,m^{-2}$", 2],
               "leaf_angle": ["Mean leaf inclination angle", r"º", 1]}

# specify range of variable values
## minimum
MIN_N_LEAF = 1.0  # From LOPEX + ANGERS average
MIN_CAB = 0.0  # From LOPEX + ANGERS average
MIN_CAR = 0.0  # From LOPEX + ANGERS average
MIN_CBROWN = 0.0  # from S2 L2B ATBD
MIN_CM = 0.0017  # From LOPEX + ANGERS average
MIN_CW = 0.000  # From LOPEX + ANGERS average
MIN_ANT = 0.0
MIN_LAI = 0.0
MIN_LEAF_ANGLE = 30.0  # from S2 L2B ATBD
MIN_HOTSPOT = 0.1  # from S2 L2B ATBD
MIN_BS = 0.50  # from S2 L2B ATBD

## maximum
MAX_N_LEAF = 3.0  # From LOPEX + ANGERS average
MAX_CAB = 110.0  # From LOPEX + ANGERS average
MAX_CAR = 30.0  # From LOPEX + ANGERS average
MAX_CBROWN = 2.00  # from S2 L2B ATBD
MAX_CM = 0.0331  # From LOPEX + ANGERS average
MAX_CW = 0.0525  # From LOPEX + ANGERS average
MAX_ANT = 40.0
MAX_LAI = 5  # from S2 L2B ATBD
MAX_LEAF_ANGLE = 80.0  # from S2 L2B ATBD
MAX_HOTSPOT = 0.5  # from S2 L2B ATBD
MAX_BS = 3.5  # from S2 L2B ATBD

prosail_bounds = {'N_leaf': (MIN_N_LEAF, MAX_N_LEAF),
                  'Cab': (MIN_CAB, MAX_CAB),
                  'Car': (MIN_CAR, MAX_CAR),
                  'Cbrown': (MIN_CBROWN, MAX_CBROWN),
                  'Cw': (MIN_CW, MAX_CW),
                  'Cm': (MIN_CM, MAX_CM),
                  'Ant': (MIN_ANT, MAX_ANT),
                  'LAI': (MIN_LAI, MAX_LAI),
                  'leaf_angle': (MIN_LEAF_ANGLE, MAX_LEAF_ANGLE),
                  'hotspot': (MIN_HOTSPOT, MAX_HOTSPOT),
                  'bs': (MIN_BS, MAX_BS)}
df_bounds = pd.DataFrame(prosail_bounds, index=['min', 'max'])
n_simulations = 40000
print(f'Setting up {n_simulations} simulations with inputs bounds:\n\n {df_bounds[OBJ_PARAM_NAMES]}')
params_orig = inv.build_prosail_database(n_simulations,
                                         param_bounds=prosail_bounds,
                                         distribution=inv.SALTELLI_DIST)
print('\nDone!')
print('Table with simulation inputs:')
pd.DataFrame(params_orig)

In [ ]:
# if you want to save the LUT generating you can can specify a directory for lut_outfile
lut_outfile = None

# spectral range
wls_sim = np.arange(400, 2501)

# number of CPUs to use to perform simulations
# (can change depending on number of CPUs in your computer)
njobs = 4

# generate LUT
rho_canopy_vec, params = inv.simulate_prosail_lut_parallel(
        njobs=njobs,
        input_dict=params_orig,
        wls_sim=wls_sim,
        rsoil_vec=soil_spectrum,
        skyl=skyl,
        sza=sza,
        vza=vza,
        psi=0,
        srf=srf,
        outfile=lut_outfile,
        calc_FAPAR=False,
        reduce_4sail=True)

print('Done!')